In [1]:
import os
import cv2
import torch
import torchvision
import numpy as np
import pandas as pd
from tqdm import tqdm
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# **Test model and make predictions**

In [2]:
device = torch.device(
    'cuda:0') if torch.cuda.is_available() else torch.device(
    'cpu')

input_folder = 'clahe_data/test'

images_test_path = f"{input_folder}"
test_images = os.listdir(images_test_path)
print(f"Validation instances: {len(test_images)}")

Validation instances: 3000


In [3]:
def get_model():
    # load an object detection model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        pretrained=True, min_size=1024)
    # one class is pneumonia, and the other is background
    num_classes = 2
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [4]:
# load a model; pre-trained on COCO
model = get_model()

model.load_state_dict(
    torch.load(
        'checkpoints/RELEASE/'
        'fasterrcnn_resnet50_fpn_pneumonia_detection_'
        'size500_dropped_clahe_e50.pth'))
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(1024,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256)
          (relu): ReLU(inplace=True)
          (downsample

In [5]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append(
            "{0:.4f} {1} {2} {3} {4}".format(
                j[0], int(j[1][0]), int(j[1][1]), int(j[1][2]), int(j[1][3])))

    return " ".join(pred_strings)

In [6]:
detection_threshold = 0.8


img_num = 0
results = []
model.eval()
with torch.no_grad():
    for i, image in tqdm(enumerate(test_images), total=len(test_images)):

        orig_image = cv2.imread(
            f"{images_test_path}/{test_images[i]}", cv2.IMREAD_COLOR)
        image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = np.transpose(image, (2, 0, 1)).astype(np.float)
        image = torch.tensor(image, dtype=torch.float).cuda()
        image = torch.unsqueeze(image, 0)

        model.eval()
        cpu_device = torch.device("cpu")

        outputs = model(image)

        outputs = [
            {k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        if len(outputs[0]['boxes']) != 0:
            for counter in range(len(outputs[0]['boxes'])):
                boxes = outputs[0]['boxes'].data.cpu().numpy()
                scores = outputs[0]['scores'].data.cpu().numpy()
                boxes = boxes[scores >= detection_threshold].astype(np.int32)
                draw_boxes = boxes.copy()
                boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] - boxes[:, 1]

            result = {
                'patientId': test_images[i].split('.')[0],
                'PredictionString': format_prediction_string(boxes, scores)
            }
            results.append(result)
        else:
            result = {
                'patientId': test_images[i].split('.')[0],
                'PredictionString': None
            }
            results.append(result)

sub_df = pd.DataFrame(results, columns=['patientId', 'PredictionString'])
print(sub_df.head())
sub_df.to_csv('submission.csv', index=False)

100%|██████████| 3000/3000 [04:25<00:00, 11.30it/s]

                              patientId       PredictionString
0  1d4413e5-f7e4-4835-9b10-53a74aaae5c2  0.8075 166 492 156 88
1  00edcd2d-d727-486d-b15a-675855e4dca3                       
2  2853a995-743e-43d8-9ea4-068ffc6bf420                       
3  1d8417fd-bdcd-455e-92eb-81a54b77173e                       
4  15443e86-b8c9-4fd2-81f2-390a6a0fffa4                       
